In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, WhiteKernel, ConstantKernel
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# 数据预处理
# 加载数据
data = pd.read_csv('./iBite_table_processed.csv')

# 特征和目标变量
feature_cols = ['head_volume', 'wing_body_ratio', 'head.w', 'head.h', 'head.l', 'body.l', 'wing.l']

# 对非数值列进行编码
non_numeric_cols = ['infraclass', 'cohort', 'order', 'suborder', 'superfamily', 
                    'family', 'subfamily', 'tribe', 'genus', 'species', 'ID', 'country']
label_encoder = LabelEncoder()
for col in non_numeric_cols:
    data_cleaned[col] = label_encoder.fit_transform(data_cleaned[col].astype(str))

# 特征工程：生成新的特征
data_cleaned['head_volume'] = data_cleaned['head.w'] * data_cleaned['head.h'] * data_cleaned['head.l']
data_cleaned['wing_body_ratio'] = data_cleaned['wing.l'] / data_cleaned['body.l']

# 对数缩放：处理数据分布不均的特征
data_cleaned['log_head_volume'] = np.log1p(data_cleaned['head_volume'])  # 加1避免取对数的负值
data_cleaned['log_body_l'] = np.log1p(data_cleaned['body.l'])

# 特征列
feature_cols = ['log_head_volume', 'log_body_l', 'wing_body_ratio']

# 特征缩放
scaler = StandardScaler()
data_cleaned[feature_cols] = scaler.fit_transform(data_cleaned[feature_cols])

X = data_cleaned[feature_cols]
y = data_cleaned['iBite']

# 训练集与测试集划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 高斯过程回归模型设置
# 核函数：尝试不同的核函数组合
kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=1.5) + WhiteKernel(noise_level=1.0)

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)
gpr.fit(X_train, y_train)

# 模型预测
y_pred_train, sigma_train = gpr.predict(X_train, return_std=True)
y_pred_test, sigma_test = gpr.predict(X_test, return_std=True)

# 模型性能评估
train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"Train MSE: {train_mse:.4f}, Train R2: {train_r2:.4f}")
print(f"Test MSE: {test_mse:.4f}, Test R2: {test_r2:.4f}")

# 可视化预测结果：测试集
plt.figure(figsize=(10, 6))
plt.errorbar(y_test, y_pred_test, yerr=sigma_test, fmt='o', alpha=0.5, label='Predicted with 95% confidence interval')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--', color='black', label='Ideal Fit')
plt.title("Gaussian Process Regression Predictions with 95% Confidence Interval")
plt.xlabel("Actual Bite Force")
plt.ylabel("Predicted Bite Force")
plt.legend()
plt.show()

# 显示优化的核函数
print(f"Optimized kernel: {gpr.kernel_}")


NameError: name 'data_cleaned' is not defined

In [2]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, Matern, WhiteKernel
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# 加载数据
data = pd.read_csv('./iBite_table_processed.csv')

# 特征和目标变量
feature_cols = ['head.w', 'head.h', 'head.l', 'th.w', 'body.l', 'wing.l', 'head_w_body_l_ratio']
X = data[feature_cols]
y = data['iBite']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 定义核函数
kernel = C(1.0) * RBF(length_scale=1.0) + WhiteKernel(noise_level=1)

# 训练模型
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True)
gpr.fit(X_train, y_train)

# 输出核函数参数
print(f"Optimized kernel: {gpr.kernel_}")

# 模型预测
y_pred, y_std = gpr.predict(X_test, return_std=True)

# 模型评估
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'GPR MSE: {mse:.4f}, R2: {r2:.4f}')

# 可视化预测结果与不确定性
# 为了简化，只绘制预测值与实际值的散点图
plt.figure(figsize=(10, 6))
plt.errorbar(y_test, y_pred, yerr=1.96 * y_std, fmt='o', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual Bite Force')
plt.ylabel('Predicted Bite Force')
plt.title('Gaussian Process Regression Predictions with 95% Confidence Interval')
plt.show()

KeyError: "['head_w_body_l_ratio'] not in index"